<a href="https://colab.research.google.com/github/moviandev/lab-neural-networks/blob/master/your-code/challenge-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/moviandev/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.


Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [17]:
# your code here
import pandas as pd

tic_tac_toe = pd.read_csv('https://raw.githubusercontent.com/moviandev/lab-neural-networks/refs/heads/master/your-code/tic-tac-toe.csv')

tic_tac_toe.head()

# See data types and non-null counts
tic_tac_toe.head()

print("\n--- Output Class Distribution ---")
# See how many wins vs. losses
print(tic_tac_toe['class'].value_counts())

print("\n--- Example Feature Values ---")
# See the unique values in one of the input columns
print(tic_tac_toe['TL'].value_counts())

# Separate the features (X) from the target (y)
X = tic_tac_toe.drop('class', axis=1)
y = tic_tac_toe['class']

# Convert the target 'y'
y = y.map({True: 1, False: 0})

# Check the result
print("\n--- Converted Target (y) ---")
print(y.head())

# Convert the features 'X' using one-hot encoding
X_encoded = pd.get_dummies(X)

# Check the result. Note how 9 columns became 27 (9 squares * 3 possible values)
print("\n--- Converted Features (X_encoded) ---")
X_encoded.head()

# Convert the X_encoded DataFrame from boolean (True/False) to integer (1/0)
X_encoded = X_encoded.astype(int)

# Check the result again (you'll see 1s and 0s instead of True/False)
print("--- Converted Features (Integers) ---")
X_encoded.head()


--- Output Class Distribution ---
class
True     626
False    332
Name: count, dtype: int64

--- Example Feature Values ---
TL
x    418
o    335
b    205
Name: count, dtype: int64

--- Converted Target (y) ---
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

--- Converted Features (X_encoded) ---
--- Converted Features (Integers) ---


,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,ML_b,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
4,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [22]:
# your code here
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Split the data into training and testing sets
# test_size=0.2 means 20% of the data will be for testing
# random_state=42 ensures you get the same "random" split every time
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Print the shapes of the new datasets to confirm it worked
print("\n--- Data Split Shapes ---")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# 1. Create the model
model = Sequential()

# 2. Add the layers
# Input layer: Must match the number of columns in X_train (27)
model.add(Dense(128, input_shape=(27,), activation='relu'))

# Hidden layer
model.add(Dense(64, activation='relu'))

# Output layer: 2 neurons (for class 0 and class 1) + softmax activation
model.add(Dense(2, activation='softmax'))

# Print a summary of the model you just built
model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
print("\n--- Model Evaluation ---")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

model.save('tic-tac-toe.model.keras')
print("\nModel saved as 'tic-tac-toe.model'")


--- Data Split Shapes ---
X_train shape: (766, 27)
y_train shape: (766,)
X_test shape: (192, 27)
y_test shape: (192,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,970 (46.76 KB)

 Trainable params: 11,970 (46.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6696 - loss: 0.6349 - val_accuracy: 0.7083 - val_loss: 0.5630
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7393 - loss: 0.5446 - val_accuracy: 0.7552 - val_loss: 0.5047
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7671 - loss: 0.4679 - val_accuracy: 0.7760 - val_loss: 0.4833
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7867 - loss: 0.4063 - val_accuracy: 0.8073 - val_loss: 0.4170
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8404 - loss: 0.3580 - val_accuracy: 0.8333 - val_loss: 0.3659
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8867 - loss: 0.3055 - val_accuracy: 0.8646 - val_loss: 0.2961
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9621 - loss: 0.2090 - val_accuracy: 0.9167 - val_loss: 0.2386
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9683 - loss: 0.1532 - val_accuracy: 0.9740 - val_loss

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [23]:
# your code here
import numpy as np
from tensorflow.keras.models import load_model

# 1. Load your saved model
try:
    loaded_model = load_model('tic-tac-toe.model.keras')
    print("Model 'tic-tac-toe.model.keras' loaded successfully.\n")
except IOError:
    print("Error: Model file 'tic-tac-toe.model.keras' not found.")
    print("Please make sure you saved the model in the previous step.")

# 2. Select 5 random rows from the test set
num_samples = 5
# Get random integer indices
random_indices = np.random.randint(0, len(X_test), num_samples)

# Use .iloc to select the rows from the pandas DataFrame/Series
sample_X = X_test.iloc[random_indices]
sample_y = y_test.iloc[random_indices]

# 3. Make predictions on these random rows
predictions = loaded_model.predict(sample_X)

# 4. Check the predictions
print("--- Checking Predictions on Random Test Data ---")

# The output of softmax is an array of probabilities for each class (e.g., [0.1, 0.9])
# We use np.argmax() to get the index (0 or 1) with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

# Convert the actual labels (sample_y) to a numpy array for easy comparison
actual_classes = sample_y.values

for i in range(num_samples):
    print(f"Sample Index: {random_indices[i]}")
    print(f"  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = {predictions[i]}")
    print(f"  -> Model Prediction: {predicted_classes[i]} ('{'Positive' if predicted_classes[i] == 1 else 'Negative'}')")
    print(f"  -> Actual Result:    {actual_classes[i]} ('{'Positive' if actual_classes[i] == 1 else 'Negative'}')")
    print(f"  -> Correct?          {predicted_classes[i] == actual_classes[i]}")
    print("-" * 30)

Model 'tic-tac-toe.keras' loaded successfully.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
--- Checking Predictions on Random Test Data ---
Sample Index: 184
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.01831322 0.98168683]
  -> Model Prediction: 1 ('Positive')
  -> Actual Result:    1 ('Positive')
  -> Correct?          True
------------------------------
Sample Index: 57
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.7415487  0.25845128]
  -> Model Prediction: 0 ('Negative')
  -> Actual Result:    0 ('Negative')
  -> Correct?          True
------------------------------
Sample Index: 96
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.9905929  0.00940713]
  -> Model Prediction: 0 ('Negative')
  -> Actual Result:    0 ('Negative')
  -> Correct?          True
------------------------------
Sample Index: 181
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.01202125 0.9879787 ]
  -> Model Prediction: 1 ('Positive')
  ->

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [25]:
# your code here
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam # Import the Adam optimizer

# --- 1. Build a Deeper Model ---
# We are adding one more Dense layer (32 neurons)
improved_model = Sequential()
improved_model.add(Dense(128, input_shape=(27,), activation='relu'))
improved_model.add(Dense(64, activation='relu'))
improved_model.add(Dense(32, activation='relu')) # <-- New hidden layer
improved_model.add(Dense(2, activation='softmax')) # Output layer remains the same

print("--- Improved Model Architecture ---")
improved_model.summary()


# --- 2. Adjust the Learning Rate ---
# Default learning rate is 0.001. Let's try a slightly smaller one.
custom_optimizer = Adam(learning_rate=0.0005)


# --- 3. Compile the Improved Model ---
improved_model.compile(optimizer=custom_optimizer, # Use our custom optimizer
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])


# --- 4. Adjust the Number of Epochs ---
# We'll train for 25 epochs instead of 10
print("\n--- Training Improved Model ---")
history_improved = improved_model.fit(X_train, y_train,
                                      epochs=25,
                                      validation_data=(X_test, y_test),
                                      verbose=1) # verbose=1 shows training progress


# --- 5. Evaluate the Improved Model ---
print("\n--- Improved Model Evaluation ---")
test_loss, test_accuracy = improved_model.evaluate(X_test, y_test)
print(f"New Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"New Test Loss: {test_loss:.4f}")


# --- 6. (Optional) Save the new model ---
improved_model.save('tic-tac-toe-improved.model.keras')
print("\nImproved model saved as 'tic-tac-toe-improved.model.keras'")

--- Improved Model Architecture ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,986 (54.63 KB)

 Trainable params: 13,986 (54.63 KB)

 Non-trainable params: 0 (0.00 B)


--- Training Improved Model ---
Epoch 1/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5206 - loss: 0.6852 - val_accuracy: 0.6510 - val_loss: 0.6177
Epoch 2/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6751 - loss: 0.5894 - val_accuracy: 0.6823 - val_loss: 0.5778
Epoch 3/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7270 - loss: 0.5601 - val_accuracy: 0.7500 - val_loss: 0.5412
Epoch 4/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7555 - loss: 0.5184 - val_accuracy: 0.7708 - val_loss: 0.5136
Epoch 5/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7614 - loss: 0.4900 - val_accuracy: 0.7708 - val_loss: 0.4896
Epoch 6/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7567 - loss: 0.4690 - val_accuracy: 0.7812 - val_loss: 0.4605
Epoch 7/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7847 - loss: 0.4312 - val_accuracy: 0.7812 - val_loss: 0.4234
Epoch 8/25
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8388 - loss: 0.3654 

**Which approach(es) did you find helpful to improve your model performance?**

In [27]:
# your answer here
import numpy as np
from tensorflow.keras.models import load_model

# 1. Load your IMPROVED saved model
try:
    # Make sure to use the new filename
    loaded_model = load_model('tic-tac-toe-improved.model.keras')
    print("Model 'tic-tac-toe-improved.model.keras' loaded successfully.\n")
except IOError:
    print("Error: Model file 'tic-tac-toe-improved.model.keras' not found.")
    print("Please make sure you saved the improved model in the previous step.")

# 2. Select 5 random rows from the test set (same as before)
num_samples = 5
random_indices = np.random.randint(0, len(X_test), num_samples)

sample_X = X_test.iloc[random_indices]
sample_y = y_test.iloc[random_indices]

# 3. Make predictions using the loaded IMPROVED model
predictions = loaded_model.predict(sample_X)

# 4. Check the predictions
print("--- Checking Predictions from IMPROVED Model ---")

predicted_classes = np.argmax(predictions, axis=1)
actual_classes = sample_y.values

for i in range(num_samples):
    print(f"Sample Index: {random_indices[i]}")
    print(f"  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = {predictions[i]}")
    print(f"  -> Model Prediction: {predicted_classes[i]} ('{'Positive' if predicted_classes[i] == 1 else 'Negative'}')")
    print(f"  -> Actual Result:    {actual_classes[i]} ('{'Positive' if actual_classes[i] == 1 else 'Negative'}')")
    print(f"  -> Correct?          {predicted_classes[i] == actual_classes[i]}")
    print("-" * 30)

Model 'tic-tac-toe-improved.model.keras' loaded successfully.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
--- Checking Predictions from IMPROVED Model ---
Sample Index: 43
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.9320708 0.0679292]
  -> Model Prediction: 0 ('Negative')
  -> Actual Result:    0 ('Negative')
  -> Correct?          True
------------------------------
Sample Index: 53
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.01440293 0.9855971 ]
  -> Model Prediction: 1 ('Positive')
  -> Actual Result:    1 ('Positive')
  -> Correct?          True
------------------------------
Sample Index: 16
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.00109666 0.9989033 ]
  -> Model Prediction: 1 ('Positive')
  -> Actual Result:    1 ('Positive')
  -> Correct?          True
------------------------------
Sample Index: 49
  -> Predicted Probabilities: [Prob(Class 0), Prob(Class 1)] = [0.9976031  0.00239687]
  -> Model Prediction: 0 ('Nega